In [332]:
import numpy as np
from qiskit import *
class ClassicalOperator:
        
        def __init__(self, shangGua, shiaGua):
            self.shangGua = np.array([[-1],[-1],[-1]])
            self.shiaGua = np.array([[-1],[-1],[-1]])
        '''
        變卦
        '''
        def bianGua(self): #shangGua: 上卦 shiaGua:下卦
            assert self.shangGua[0,0] != -1
            print("[classical變卦]")
            shangGua = np.where((self.shangGua==0)|(self.shangGua==1), self.shangGua^1, self.shangGua)
            shiaGua = np.where((self.shiaGua==0)|(self.shiaGua==1), self.shiaGua^1, self.shiaGua)
            return(shangGua, shiaGua) 
        '''
        互卦
        '''
        def huGua(self):
            assert self.shangGua[0,0] != -1
            print("[classical互卦]")
            shangGua = np.array([ [self.shangGua[1,0]],
                                  [self.shangGua[2,0]],
                                  [self.shiaGua[0,0]]])
            shiaGua = np.array([ [self.shangGua[2,0]],
                                 [self.shiaGua[0,0]],
                                 [self.shiaGua[2,0]]])
            return(shangGua, shiaGua)
        '''
        綜卦
        '''
        def tzungGua(self):
            assert self.shangGua[0,0] != -1
            print("[classical綜卦]")
            shangGua = np.flip(self.shangGua)
            shiaGua = np.flip(self.shiaGua)
            return(shangGua, shiaGua)
        '''
        正卦
        '''
        def zhenGua(self):
            print("[classical正卦]")
            self.getGua("shangGua")
            self.getGua("shiaGua")
            return(self.shangGua, self.shiaGua)
        def getGua(self,mode="shangGua"): #Two mode: shanGua or shiaGua
            qr = QuantumRegister(3)
            cr = ClassicalRegister(3)
            circuit = QuantumCircuit(qr, cr)
            circuit.h(qr)
            circuit.measure(qr, cr)
            %matplotlib inline 
            circuit.draw(output='mpl')
            backend = Aer.get_backend('qasm_simulator')
            job = execute(circuit, backend, shots=1)
            result = job.result().get_counts(circuit)
            for x,y in result.items():
                Gua = x
                #print(Gua)
            if(mode=="shangGua"):
                self.shangGua[0,0] = Gua[2]
                self.shangGua[1,0] = Gua[1]
                self.shangGua[2,0] = Gua[0]
                #print(self.shangGua)
            elif(mode=="shiaGua"):
                self.shiaGua[0,0] = Gua[2]
                self.shiaGua[1,0] = Gua[1]
                self.shiaGua[2,0] = Gua[0]
                #print(self.shiaGua)
            else:
                print("mode error")



In [333]:
classicalOp = ClassicalOperator(a,b)

In [334]:
classicalOp.zhenGua()
classicalOp.bianGua()
classicalOp.huGua()
classicalOp.tzungGua()

[classical正卦]
[classical變卦]
[classical互卦]
[classical綜卦]


(array([[0],
        [1],
        [1]]),
 array([[1],
        [0],
        [0]]))

In [335]:
class QuantumOperator:
        
        def __init__(self, shangGua, shiaGua):
            self.shangGua = np.array([[-1],[-1],[-1]])
            self.shiaGua = np.array([[-1],[-1],[-1]])
            self.shangGua_zhen = np.array([[-1],[-1],[-1]])
            self.shiaGua_zhen = np.array([[-1],[-1],[-1]])
            self.crBuffer = []
        '''
        變卦
        '''
        def bianGua(self): #shangGua: 上卦 shiaGua:下卦
            print("[Quantum變卦]")
            circ = QuantumCircuit(6)
            for i in range(5):
                circ.h(i)
                circ.x(i)
                circ.cx(i,5)
        
            circ.measure_all()
    
            simulator = Aer.get_backend('qasm_simulator')
            job = execute(circ, simulator, shots=1)
            result = job.result().get_counts(circ)
            self.__resultToGua(result)
            return(self.shangGua, self.shiaGua)
        '''
        互卦
        '''
        def huGua(self):
            print("[Quantum互卦]")
            qr = QuantumRegister(6)
            cr = ClassicalRegister(8) # cr[6]: for qr[0] cr[7]: for qr[5]
            circ = QuantumCircuit(qr, cr)
            for i in range(5):
                circ.h(i)
                circ.cx(i,5)
    
            circ.measure(qr[1], cr[0])
            circ.measure(qr[2], cr[1])
            circ.measure(qr[3], cr[2])
            circ.measure(qr[2], cr[3])
            circ.measure(qr[3], cr[4])
            circ.measure(qr[4], cr[5])

            circ.measure(qr[0], cr[6])
            circ.measure(qr[1], cr[7])
            simulator = Aer.get_backend('qasm_simulator')
            job = execute(circ, simulator, shots=1)
            result = job.result().get_counts(circ)
            
            for x,y in result.items():
                Gua = x
            self.shangGua[0,0]=Gua[7]
            self.shangGua[1,0]=Gua[6]
            self.shangGua[2,0]=Gua[5]
            self.shiaGua[2,0]=Gua[4]
            self.shiaGua[1,0]=Gua[3]
            self.shiaGua[0,0]=Gua[2]

            self.crBuffer.append(Gua[1])# q[0]
            self.crBuffer.append(Gua[0])# q[6]

            return(self.shangGua, self.shiaGua)
        '''
        綜卦
        '''
        def tzungGua(self):
            print("[classical綜卦]")
            circ = QuantumCircuit(6)
            for i in range(5):
                circ.h(i)
                circ.x(i)
                circ.cx(i,5)
            circ.swap(0,2)
            circ.swap(3,5)
       
            circ.measure_all()
    
            simulator = Aer.get_backend('qasm_simulator')
            job = execute(circ, simulator, shots=1)
            result = job.result().get_counts(circ)
            self.__resultToGua(result)
            return(self.shangGua, self.shiaGua)
        '''
        正卦
        '''
        def zhenGua(self):
            print("[Quantum正卦]")
            circ = QuantumCircuit(6) # 6 qubits
            for i in range(5):
                circ.h(i)
                circ.cx(i,5)
            circ.measure_all()
            simulator = Aer.get_backend('qasm_simulator')
            job = execute(circ, simulator, shots=1)
            result = job.result().get_counts(circ)
            self.__resultToGua(result)
            return(self.shangGua, self.shiaGua)
        def __resultToGua(self, result): #string to shanGua & shiaGua
            for x,y in result.items():
                Gua = x
            self.shangGua[0,0]=Gua[5]
            self.shangGua[1,0]=Gua[4]
            self.shangGua[2,0]=Gua[3]
            self.shiaGua[2,0]=Gua[2]
            self.shiaGua[1,0]=Gua[1]
            self.shiaGua[0,0]=Gua[0]
        def transZhenGua(self, mode="huGua"): # mode = "huGua" or "bianGua" or "tzungGua" 
            if(mode=="huGua"):
                self.shangGua_zhen[0,0] = self.crBuffer[0]
                self.shangGua_zhen[1,0] = self.shangGua[0,0]
                self.shangGua_zhen[2,0] = self.shangGua[1,0]

                self.shiaGua_zhen[0,0] = self.shiaGua[1,0]
                self.shiaGua_zhen[1,0] = self.shiaGua[2,0]
                self.shiaGua_zhen[2,0] = self.crBuffer[1]

                return(self.shangGua_zhen, self.shiaGua_zhen)
            elif(mode=="bianGua"):
                self.shangGua_zhen = np.where((self.shangGua==0)|(self.shangGua==1), self.shangGua^1, self.shangGua)
                self.shiaGua_zhen = np.where((self.shiaGua==0)|(self.shiaGua==1), self.shiaGua^1, self.shiaGua)
                return(self.shangGua_zhen, self.shiaGua_zhen)
            elif(mode=="tzungGua"):
                self.shangGua_zhen = np.flip(self.shangGua)
                self.shiaGua_zhen = np.flip(self.shiaGua)
                return(self.shangGua_zhen, self.shiaGua_zhen)
            else:
                return -1 #error

In [336]:
quantumOp = QuantumOperator(a,b)
quantumOp.zhenGua()
quantumOp.bianGua()
quantumOp.huGua()
quantumOp.transZhenGua("tzungGua")

[Quantum正卦]
[Quantum變卦]
[Quantum互卦]


(array([[1],
        [1],
        [1]]),
 array([[1],
        [1],
        [1]]))

In [340]:
zdic = {}
with open('D:/QiskitHackathon/Divination/64.txt', encoding = 'utf8') as zfile:
    for line in zfile:
        z = line.split(' ', 1)
        key = z[0]
        value = z[1].split('，',1)
        zdic[key] = value
        
        
for x,y in zdic.items():
    print('%s %s %s' %(x,y[0],y[1])) 

111111 乾 周易六十四卦之首。上下皆由相同的乾卦組成，其六個爻皆為陽。通稱「乾為天」。 代表「天」的形象。置於六十四卦之首、其次是象徵「地」的坤卦，序卦傳：天地定位、萬物生焉。

000000 坤 周易六十四卦中第二卦。上下皆是由坤卦組成，六個爻皆是陰爻。通稱為「坤為地」。象徵「大地」、與天共同孕育萬物之生成。

010001 屯 周易六十四卦中第三卦。外卦（上卦）為坎、內卦（下卦）為震。因為上卦是代表水的坎卦、下卦是代表雷的震卦，所以通稱為「水雷屯」。 天地定位後萬物生長，屯卦有「盈」「萬物始生」之意。

100010 蒙 周易六十四卦中第四卦。外卦（上卦）為艮、內卦（下卦）為坎。因為上卦是代表山的艮卦、下卦是代表水的坎卦，故通稱「山水蒙」。象徵萬物初生，「蒙昧」的狀態。

010111 需 周易六十四卦中第五卦。外卦（上卦）為坎、內卦（下卦）為乾。因為代表水的坎卦在上、代表天的乾卦在下，故通稱為「水天需」。依序卦傳的解釋需為「飲食之道」，指萬物啟蒙後的養育。

111010 訟 周易六十四卦中第六卦。外卦（上卦）乾、內卦（下卦）坎。因為代表天的乾卦在上、代表水的坎卦在下，是故通稱為「天水訟」。依序卦傳的解釋，為了飲食生活的「需」求，開始會有爭執，是為「爭訟」，是以排序在需卦之後。

000010 師 周易六十四卦中第七卦。外卦（上卦）坤、內卦（下卦）坎、因為代表地的坤卦在上、代表水的坎卦在下，所以通稱為「地水師」。師為軍隊之意、因為群眾的爭執，減變成「興兵為師」的狀況。

010000 比 周易六十四卦中第八卦。外卦（上卦）坎、內卦（下卦）坤。因為代表水的坎卦在上、代表地的坤卦在下，故通稱為「水地比」。比為比鄰，親近友好之意，起兵興師後同群之人為「比」。

110111 小畜 周易六十四卦中第九卦。外卦（上卦）巽，內卦（下卦）乾，通稱「風天小畜」。小畜有集合之意，人們親近後開始集合。

111011 履 周易六十四卦中第十卦。外卦（上卦）乾，內卦（下卦）兌，通稱「天澤履」。履為踩踏之意，序卦傳另云：履者禮也。

000111 泰 周易六十四卦中第十一卦。外卦（上卦）坤，內卦（下卦）乾，通稱「地天泰」。泰為通達之意。

111000 否 周易六十四卦中第十二卦。外卦（上卦）乾，內卦（下卦）坤，通稱「天地否」。否為閉「塞」之意。

111101 同人 周易六十